<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1">Description</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2">Setup</a></span></li><li><span><a href="#State-Functions" data-toc-modified-id="State-Functions-3">State Functions</a></span></li><li><span><a href="#Step-Functions" data-toc-modified-id="Step-Functions-4">Step Functions</a></span></li><li><span><a href="#Simulation-Function" data-toc-modified-id="Simulation-Function-5">Simulation Function</a></span></li><li><span><a href="#Metrics-and-Visualizations" data-toc-modified-id="Metrics-and-Visualizations-6">Metrics and Visualizations</a></span></li><li><span><a href="#Run-Simulations" data-toc-modified-id="Run-Simulations-7">Run Simulations</a></span></li></ul></div>

## Description

Using SimPy, write a process simulation that includes waiting time (discrete event simulation).  You may use any topic of interest to you.  Write the simulation and all of the following in Jupyter.
Each element is worth 5 points and will be graded using the rubric shown here.

1.  State the problem and its significance.
2.  Provide a flow-chart model. 
3.  Simulate the process for the appropriate number of iterations (justify)
4.  Justify the validity of the model and discuss how you verified it.
5.  State  your conclusions/ findings from the model. 
6.  Generate appropriate graphs (more than one) to illustrate the results and provide a PowerPoint presentation to share with your colleagues.  Post this to the discussion. 


## Setup

In [1]:
# Configure Jupyter so figures appear in the notebook
import matplotlib.pyplot as plt

# Configure Jupyter to display the assigned value after an assignment
#%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [2]:
#Variables for TimeSeries Tracking and Reporting
results_entrance1 = TimeSeries()
results_entrance2 = TimeSeries()
results_parkingzone1 = TimeSeries()
results_parkingzone2 = TimeSeries()

results_no_cart_for_customer_entrance1 = TimeSeries()
results_no_cart_for_customer_entrance2 = TimeSeries()

## State Functions

In [3]:
#Create the Carts State object
def create_new_carts_object(c, p1):
    ''' initial state is total carts passed in. Then use p1 to set the percent of customers that enter via
    Entrance 1 (groceries) and Entrance 2
    '''
    
    entrance1_count = c * p1
    entrance2_count = c - entrance1_count
    carts = State(entrance1=entrance1_count, entrance2=entrance2_count, instore=0, parkingzone1=0, parkingzone2=0)
    
    return carts

In [4]:
#Create the Customers State object
def create_new_customers_object():
    ''' initial state is 0 for entrance 1 and entrance 2. This tracks the number of customers that don't have a
    cart waiting when they arrive at an entrance.  This state is reset every step (hour)
    '''
    customers = State(no_cart_for_customer_entrance1=0, no_cart_for_customer_entrance2=0)
    
    return customers

In [5]:
def entrance1_to_instore():
    """Shopper enters store and grabs a cart from entrance 1
    
    state: carts State object
    """
    if carts.entrance1 <= 0:
        customers.no_cart_for_customer_entrance1 += 1
    else:
        carts.entrance1 -= 1
        carts.instore += 1

In [6]:
def entrance2_to_instore():
    """Shopper enters store and grabs a cart from entrance 2
    
    state: carts State object
    """
    if carts.entrance2 <= 0:
        customers.no_cart_for_customer_entrance2 += 1
    else:
        carts.entrance2 -= 1
        carts.instore += 1

In [7]:
def exit_store_leave_cart_at_entrance1():
    """Shopper departs store and returns the cart to the entrance they entered
    
    state: carts State object
    """
    if carts.instore == 0:
        return
    carts.instore -= 1
    carts.entrance1 += 1

In [8]:
def exit_store_leave_cart_in_parkingzone1():
    """Shopper departs store and leaves the cart in the parking lot
    
    state: carts State object
    """
    if carts.instore == 0:
        return
    carts.instore -= 1
    carts.parkingzone1 += 1

In [9]:
def exit_store_leave_cart_at_entrance2():
    """Shopper departs store and returns the cart to the entrance they entered
    
    state: carts State object
    """
    if carts.instore == 0:
        return
    carts.instore -= 1
    carts.entrance2 += 1

In [10]:
def exit_store_leave_cart_in_parkingzone2():
    """Shopper departs store and leaves the cart in the parking lot
    
    state: carts State object
    """
    if carts.instore == 0:
        return
    carts.instore -= 1
    carts.parkingzone2 += 1

In [11]:
def parkingzone1_to_entrance1():
    """Cart Return employee returns cart to the closest entrance
    
    state: carts State object
    """
    if carts.parkingzone1 == 0:
        return
    carts.parkingzone1 -= 1
    carts.entrance1 += 1

In [12]:
def parkingzone2_to_entrance2():
    """Cart Return employee returns cart to the closest entrance
    
    state: carts State object
    """
    if carts.parkingzone2 == 0:
        return
    carts.parkingzone2 -= 1
    carts.entrance2 += 1

## Step Functions

In [13]:
#Assume that 60% of the customers enter via Entrance 1 and 40% enter via Entrance 2
def step_enter_the_store(p1):
    """Simulate a shopper entering the store
    
    state: carts State object
    p1: probability of a shopper entering entrance 1
    """
    if flip(p1):
        entrance1_to_instore()
    else:
        entrance2_to_instore()

In [14]:
#p1 represents the assumption that customers exit the store via the same entrance they entered (60% Entrance1,40% Entrance2)

#p2 represents that when exiting, 10% of the customers return cart to entrance and 90% leave cart in parking lot

def step_exit_the_store(p1,p2):
    """Simulate a shopper leaving the store
    
    state: carts State object
    p1: probability of a shopper leaving via entrance 1
    p2: probability of a shopper returning the cart to the entrance, versus taking cart out to parking lot
    """
    if flip(p1):
        if flip(p2):
            exit_store_leave_cart_at_entrance1()
        else:
            exit_store_leave_cart_in_parkingzone1()
        
    else:
        if flip(p2):
            exit_store_leave_cart_at_entrance2()
        else:
            exit_store_leave_cart_in_parkingzone2()

In [15]:
#Return carts from parking lot zones to entrances
#Cart pushers is the number of employees assigned to return carts and cart pusher volume is the number of carts
#per hour that they can return

def return_carts_to_entrance(cart_pushers_parking_zone1, cart_pushers_parking_zone2, cart_pusher_volume):
    """Represent a cart returner returning carts from parking lot to nearest entrance
    
    state: bikeshare State object
    cart pushers: number of employees returning carts to an entrance
    cart pusher volume: number of carts per step (hour) that a cart pusher can return
    """
    #maximum total carts that can be returned
    max_total_carts_parking_zone1 = cart_pushers_parking_zone1 * cart_pusher_volume
    max_total_carts_parking_zone2 = cart_pushers_parking_zone2 * cart_pusher_volume
    
    #actual carts in parking lot
    actual_carts_parkingzone1 = carts.parkingzone1
    actual_carts_parkingzone2 = carts.parkingzone2
    
    #decide between max and actual
    value_entrance1 = 0
    value_entrance2 = 0
    
    if actual_carts_parkingzone1 <= max_total_carts_parking_zone1:
        value1 = actual_carts_parkingzone1
    else:
        value1 = max_total_carts_parking_zone1
    
    if actual_carts_parkingzone2 <= max_total_carts_parking_zone2:
        value2 = actual_carts_parkingzone2
    else:
        value2 = max_total_carts_parking_zone2
    
    #carts returned to entrance
    carts.entrance1 += value1
    carts.parkingzone1 -= value1
    
    carts.entrance2 += value2
    carts.parkingzone2 -= value2

## Simulation Function

The function below runs the simulationEach step corresponds to an hour in the 24 hours that Meijer is typically open each day.
Within each hour an average number of customers enter the store

In [16]:
def run_simulation():
    for i in range(global_hours_store_is_open):
        
        #cart pushers
        return_carts_to_entrance(global_cart_pushers_parking_zone1,
                                 global_cart_pushers_parking_zone2, global_cart_pusher_volume)
        
        #shoppers
        #customers = customers_no_cart_available()
        for j in range(global_shoppers_per_hour):
            step_enter_the_store(global_p1)
            step_exit_the_store(global_p1,global_p2)
            
        results_entrance1[i] = carts.entrance1
        results_entrance2[i] = carts.entrance2
        results_parkingzone1[i] = carts.parkingzone1
        results_parkingzone2[i] = carts.parkingzone2
        
        results_no_cart_for_customer_entrance1[i] = customers.no_cart_for_customer_entrance1
        results_no_cart_for_customer_entrance2[i] = customers.no_cart_for_customer_entrance2

Set the initial state (midnight) and display the cart count values

## Metrics and Visualizations

In [17]:
def decorate_customers3(x,y,intitle,a):
    a[x,y].set_title(intitle)
    a[x,y].set_xlabel('steps (hours of day)')
    a[x,y].set_ylabel('Number of Customers')
    
def decorate_carts3(x,y,intitle,a):
    a[x,y].set_title(intitle)
    a[x,y].set_xlabel('steps (hours of day)')
    a[x,y].set_ylabel('Number of Carts')

In [18]:
#Summary plot
def summary_plot(outputfilename):
    fig, a =  plt.subplots(3,2, figsize=(10,10))
    fig.tight_layout(pad=3.0)

    #Customers
    a[0,0].plot(results_no_cart_for_customer_entrance1)
    decorate_customers3(0,0,'Customers with no cart available at Entrance 1',a)

    a[0,1].plot(results_no_cart_for_customer_entrance2)
    decorate_customers3(0,1,'Customers with no Cart Available at Entrance 2',a)

    #Carts
    a[1,0].plot(results_entrance1)
    decorate_carts3(1,0,'Entrance 1 Carts',a)

    a[1,1].plot(results_entrance2)
    decorate_carts3(1,1,'Entrance 2 Carts',a)

    #Parking Lot
    a[2,0].plot(results_parkingzone1)
    decorate_carts3(2,0,'Parking Zone 1',a)

    a[2,1].plot(results_parkingzone2)
    decorate_carts3(2,1,'Parking Zone 2',a)

    #plt.show()
    plt.suptitle(outputfilename)
    fig.savefig(outputfilename + '.png')
    plt.close(fig)

In [19]:
#Metrics
def summary_metrics(c):
    print('**************************************************************************')
    print(f'Summary metrics for', c, ' carts')
    print()
    #Descriptive statistics for Customers
    print(f'Total customers over 24 hours with no cart at Entrance 1:', max(results_no_cart_for_customer_entrance1))
    print(f'Total customers over 24 with no cart at Entrance 2:', max(results_no_cart_for_customer_entrance2))
    print()

    #Descriptive statistics for Carts at Entrances
    print(f'Minimum cart count for Entrance 1 at end of hour:', min(results_entrance1))
    print(f'Minimum cart count for Entrance 2 at end of hour:', min(results_entrance2))
    print()

    #Descriptive statistics for Parking Lot
    print(f'Minimum cart count for Parking Zone 1 at end of hour:', min(results_parkingzone1))
    print(f'Minimum cart count for Parking Zone 2 at end of hour:', min(results_parkingzone2))

## Run Simulations

In [20]:
#Globals
global_hours_store_is_open = 24
global_total_shoppers = 5313 #avg shoppers per day on Saturday (busiest day)
global_shoppers_per_hour = int(global_total_shoppers/global_hours_store_is_open)

global_cart_pushers_parking_zone1 = 1.5
global_cart_pushers_parking_zone2 = 1.5
global_cart_pusher_volume = 100

global_p1 = 0.6 #percent of shoppers that enter entrance 1 versus entrance 2
global_p2 = 0.1 #percent of shoppers that return their cart to an entrance, versus to the parking lot

In [21]:
#Run the simulation for varying numbers of cart counts
input_cart_counts = [450, 400, 350, 300, 250, 200, 150]

for c in input_cart_counts:
    carts = create_new_carts_object(c, global_p1)
    customers = create_new_customers_object()
    run_simulation()
    
    outputfilename = 'Plots for ' + str(c) + ' carts'
    summary_plot(outputfilename)
    
    summary_metrics(c)


**************************************************************************
Summary metrics for 450  carts

Total customers over 24 hours with no cart at Entrance 1: 0
Total customers over 24 with no cart at Entrance 2: 0

Minimum cart count for Entrance 1 at end of hour: 112.0
Minimum cart count for Entrance 2 at end of hour: 84.0

Minimum cart count for Parking Zone 1 at end of hour: 103.0
Minimum cart count for Parking Zone 2 at end of hour: 70.0
**************************************************************************
Summary metrics for 400  carts

Total customers over 24 hours with no cart at Entrance 1: 0
Total customers over 24 with no cart at Entrance 2: 0

Minimum cart count for Entrance 1 at end of hour: 86.0
Minimum cart count for Entrance 2 at end of hour: 44.0

Minimum cart count for Parking Zone 1 at end of hour: 104.0
Minimum cart count for Parking Zone 2 at end of hour: 66.0
**************************************************************************
Summary metrics for 